<center><img src="images/header.png"></center>

<h1><center>Алгоритмы интеллектуальной обработки больших объемов данных</center></h1>
<hr>
<h2><center>Деревья решений, случайные леса (практика)</center></h2>

**Срок сдачи:** 3 апреля 2017, 06:00 <br> 
**Штраф за опоздание:** -2 балла после 06:00 3 апреля, -4 балла после 06:00 10 апреля, -6 баллов после 06:00 17 апреля

При отправлении ДЗ указывайте фамилию в названии файла<br>

Не забудте создать задание в Slack через /todo.

Используйте данный Ipython Notebook при оформлении домашнего задания.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

# Практика

### Качество вина (10 баллов)

Давайте все-таки разберемся с [вином](https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv)!


#### Задание (1 балл)
* Загрузите данные, выполните преобразование целевой переменной: 
    * quality > 5 -> 1
    * quality <= 5 -> 0
* Разделите выборку на обучающую и контрольную в пропорции 80/20

In [3]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')

URLError: <urlopen error [Errno 10061] Подключение не установлено,>

In [ ]:
df_wine.head()

In [ ]:
df_wine.columns

In [ ]:
df_wine.loc[:, 'quality'] = (df_wine.loc[:, 'quality'] > 5).astype(int)

In [ ]:
X, y = df_wine.iloc[:,:-1].values, df_wine.iloc[:,-1].values

In [ ]:
from sklearn.cross_validation import train_test_split

In [ ]:
# partition
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

#### Задание (2 балла)

Сфокусируемся на одном гиперпараметре деревьев решений - максимальной глубине.

Подберите наилучшую глубину `d` дерева с помошью 
* Усредненной оценки качества roc-auc на кросс-валидации при различных `d`
* Валидационных кривых

In [ ]:
from sklearn.tree import DecisionTreeClassifier

try:
    from sklearn.model_selection import cross_val_score
except ImportError:
    from sklearn.cross_validation import cross_val_score

try:
    from sklearn.model_selection import validation_curve
except ImportError:
    from sklearn.learning_curve import validation_curve

In [ ]:
scores = []
for d in range(1,20):
    model = DecisionTreeClassifier(max_depth=d)
    scores.append(cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc').mean())
print scores

In [ ]:
max(scores)

In [ ]:
plt.plot(range(1,20), scores)

In [ ]:
# При мере качества roc-auc виден явный пик на глубине 4
# Если выбрать меру качества accuracy

In [ ]:
scores = []
for d in range(1,20):
    model = DecisionTreeClassifier(max_depth=d)
    scores.append(cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean())
#print scores

In [ ]:
plt.plot(range(1,20), scores)

In [ ]:
# Воспользуемся validation_curve 
# В результате возвращаются мера качества как на тестовой части каждого из фолдов, так и на обучающей части

In [ ]:
model = DecisionTreeClassifier(random_state=123) 
train_scores, test_scores = validation_curve(model, X_train, y_train, 'max_depth', 
                                            range(1, 20), n_jobs=-1, cv=5, scoring='roc_auc')

In [ ]:
train_mean = train_scores.mean(axis=1) # по строкам
test_mean = test_scores.mean(axis=1) 
train_std = train_scores.std(axis=1) 
test_std = test_scores.std(axis=1)

In [ ]:
d = range(1,20) 

plt.plot(d, train_mean, color='b') 
plt.plot(d, test_mean) 

plt.fill_between(d, train_mean-train_std, train_mean+train_std, color='b', alpha=0.3) 
plt.fill_between(d, test_mean-test_std, test_mean+test_std, color='r', alpha=0.3)

In [ ]:
# Виден тот же самый пик при глубине 4

In [ ]:
# Возьмем метрику accuracy

In [ ]:
model = DecisionTreeClassifier(random_state=123) 
train_scores, test_scores = validation_curve(model, X_train, y_train, 'max_depth', 
                                            range(1, 20), n_jobs=-1, cv=5, scoring='accuracy')

In [ ]:
train_mean = train_scores.mean(axis=1) # по строкам
test_mean = test_scores.mean(axis=1) 
train_std = train_scores.std(axis=1) 
test_std = test_scores.std(axis=1)

In [ ]:
d = range(1,20) 

plt.plot(d, train_mean, color='b') 
plt.plot(d, test_mean) 

plt.fill_between(d, train_mean-train_std, train_mean+train_std, color='b', alpha=0.3) 
plt.fill_between(d, test_mean-test_std, test_mean+test_std, color='r', alpha=0.3)

In [ ]:
# На той глубине, где accuracy был наибольший (около 14) ошибка на трейновой части минимальная, есть очень сильное переобучение. 

In [ ]:
# Вывод: Основываясь на том, что при метрике roc-auc виден пик при глубине 4, и на том, что переобучение по 
# accuracy не такое сильное при max_depth=4, а качество будет лучше, чем в окрестности, то выберем глубину 4

#### Задание (1 балл)

Отсортируйте признаки по важности. Вектор с важностью признаков можно получить с помощью `model.feature_importances_`

In [ ]:
model = DecisionTreeClassifier(random_state=123, max_depth=4) 
model.fit(X_train, y_train)

importances = model.feature_importances_

In [ ]:
importance_dict = dict(zip(df_wine.columns, importances))
importance_dict

In [ ]:
sorted(importance_dict, key=importance_dict.get)[::-1]

In [ ]:
# Sorted tags by importance. The main tag is alcohol, less important tags:
# free sulfur dioxide, citric acid, fixed acidity, chlorides

#### Задание (2 балла)
* Получите предсказания на тестовой выборке
* Постройте ROC кривые для обучающей и тестовой выборок
* Посчитайте остальные меры качества классификации при пороге в `0.5`

In [ ]:
# Prediction testing
predictions_test = model.predict(X_test)
probability_predictions_test = model.predict_proba(X_test)

In [ ]:
# Prediction teaching
predictions_train = model.predict(X_train)
probability_predictions_train = model.predict_proba(X_train)

In [ ]:
from sklearn import metrics

In [ ]:
# For test sapmle, label 1
fpr, tpr, thresholds = metrics.roc_curve(y_test, probability_predictions_test[:,1], pos_label=1)

In [ ]:
%pylab inline
pylab.plot(fpr, tpr, label = 'model') 
pylab.plot([0, 1], [0, 1], '--', color = 'grey', label = 'random') 
pylab.xlim([-0.05, 1.05]) 
pylab.ylim([-0.05, 1.05]) 
pylab.xlabel('False Positive Rate') 
pylab.ylabel('True Positive Rate') 
pylab.title('ROC curve') 
pylab.legend(loc = "lower right")

In [ ]:
# For teaching sample, label 0
fpr, tpr, thresholds = metrics.roc_curve(y_train, probability_predictions_train[:,0], pos_label=0)

In [ ]:
pylab.plot(fpr, tpr, label = 'model') 
pylab.plot([0, 1], [0, 1], '--', color = 'grey', label = 'random') 
pylab.xlim([-0.05, 1.05]) 
pylab.ylim([-0.05, 1.05]) 
pylab.xlabel('False Positive Rate') 
pylab.ylabel('True Positive Rate') 
pylab.title('ROC curve') 
pylab.legend(loc = "lower right")

In [ ]:
from sklearn import metrics
from sklearn.preprocessing import Binarizer

In [ ]:
binz = Binarizer(threshold=0.5).fit(probability_predictions_test)  # fit does nothing

In [ ]:
res = binz.transform(probability_predictions_test)

In [ ]:
binz = Binarizer(threshold=0.5).fit_transform(probability_predictions_test)

In [ ]:
metrics.accuracy_score(y_test, binz[:,1]) # accuracy_score for test

In [ ]:
metrics.precision_score(y_test, binz[:,1], pos_label=1) # precision label 0 

In [ ]:
metrics.precision_score(y_test, binz[:,1], pos_label=0) # precision label 1

In [ ]:
metrics.recall_score(y_test, binz[:,1], pos_label=1) # recall для 1 класса

In [ ]:
metrics.recall_score(y_test, binz[:,1], pos_label=0) # recall для 0 класса 

In [ ]:
metrics.f1_score(y_test, binz[:,1], pos_label=1) # f1 для 1 класса 

In [ ]:
metrics.f1_score(y_test, binz[:,1], pos_label=0) # f1 для 0 класса 

In [ ]:
print metrics.classification_report(y_test, binz[:,1])

In [ ]:
metrics.roc_auc_score(y_test, binz[:,1])

In [ ]:
metrics.average_precision_score(y_test, binz[:,1])

#### Задание (1 балл)
Перейдем к модели случайного леса. Зафиксируем некоторую глубину дерева (можно звять оптимальное с предыдущих заданий).

Сравните качество работы 
* Простого дерева решений
* Бэггинга над 10,20,...,100 деревьями решений
* Случайного леса с 10,20,...,100 деревьями решений

Получите график, у которого по оси X откладывается количество деревьев, а по оси Y - качество классификации.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
simple_model = DecisionTreeClassifier(random_state=123, max_depth=4) 

In [ ]:
simple_model.fit(X_train, y_train)
simple_model_lables = simple_model.predict_proba(X_test)

In [ ]:
metrics.roc_auc_score(y_test, simple_model_lables[:,1])

In [ ]:
# random forest
scores_train = []
scores_test = []

for d in range(10, 100, 10):
    
    model = RandomForestClassifier(n_estimators=d, max_depth=4, random_state=123)
    scores_train.append(cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc').mean())
    model.fit(X_train, y_train)
    labels = model.predict_proba(X_test)
    res_lables = metrics.roc_auc_score(y_test, labels[:,1])
    scores_test.append(res_lables)

In [ ]:
plt.plot(range(10, 100, 10), scores_train)

In [ ]:
plt.plot(range(10, 100, 10), scores_test)

In [ ]:
# bagging
scores_test = []
scores_train = []

for d in range(10, 100, 10):
    
    model = BaggingClassifier(n_estimators=d, random_state=123)
    scores_train.append(cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc').mean())
    model.fit(X_train, y_train)
    labels = model.predict_proba(X_test)
    res_lables = metrics.roc_auc_score(y_test, labels[:,1])
    scores_test.append(res_lables)

In [ ]:
plt.plot(range(10, 100, 10), scores_train)

In [ ]:
plt.plot(range(10, 100, 10), scores_test)

####  Задание (3 балла)
* С помощью `GridSearchCV` или `RandomSearchCV` подберите наиболее оптимальные параметры для случайного леса.
* Для этих параметров сравните средние результаты по кросс-валидации и качество на контрольной выборке

In [ ]:
from sklearn import grid_search

In [ ]:
clf = RandomForestClassifier(random_state=123)

In [ ]:
clf.get_params().keys()

In [ ]:
param_grid = {
    'max_depth' : range(1, 20),
    'n_estimators' : range(1, 100, 10),   
}

In [ ]:
from sklearn import cross_validation

In [ ]:
grid_cv = grid_search.GridSearchCV(clf, param_grid, scoring='roc_auc')

In [ ]:
grid_cv.fit(X_train, y_train)

In [ ]:
grid_cv.best_estimator_

In [ ]:
print grid_cv.best_score_
print grid_cv.best_params_

In [ ]:
#grid_cv.grid_scores_[:10]

In [ ]:
model = RandomForestClassifier(random_state=123, n_estimators=91, max_depth=15)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
labels_proba = model.predict_proba(X_test)
score_proba = metrics.roc_auc_score(y_test, labels_proba[:,1])
score_proba

In [ ]:
score_train = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc').mean()
score_train

In [ ]:
labels = model.predict(X_test)
score = metrics.accuracy_score(y_test, labels)
score

In [ ]:
score_train = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()
score_train

In [ ]:
labels = model.predict(X_test)
score = metrics.precision_score(y_test, labels)
score

In [ ]:
score_train = cross_val_score(model, X_train, y_train, cv=5, scoring='precision').mean()
score_train

In [ ]:
labels = model.predict(X_test)
score = metrics.recall_score(y_test, labels)
score

In [ ]:
score_train = cross_val_score(model, X_train, y_train, cv=5, scoring='recall').mean()
score_train

In [ ]:
labels = model.predict(X_test)
score = metrics.f1_score(y_test, labels)
score

In [ ]:
score_train = cross_val_score(model, X_train, y_train, cv=5, scoring='f1').mean()
score_train